In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

from sklearn.preprocessing import OneHotEncoder
import gensim 
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors


Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [4]:
# load
in_filename = './data/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [5]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [6]:
print(doc)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

## embedding style

#### one hot encoding

In [8]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [9]:
print(X.shape)

(219583, 50)


In [10]:
print(y.shape)

(219583, 10650)


In [11]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            532500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10650)             1075650   
Total params: 1,759,050
Trainable params: 1,759,050
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            532500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10650)             1075650   
Total params: 1,759,050
Trainable params: 1,759,050
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)
# fit model glove
# model.fit(padded_docs, max_length, batch_size=128, epochs=100)

c:\users\dell\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/100
 24448/219583 [==>...........................] - ETA: 6:48 - loss: 6.8101 - accuracy: 0.0665

KeyboardInterrupt: 

In [ ]:
# save the model to file
model.save('Languege_model_one_hot.h5')

In [ ]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))